In [1]:
from pyspark.sql import SparkSession, functions as F

# Cell to create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 ASSIGNMENT 1 DUSTIN")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/08/19 00:23:03 WARN Utils: Your hostname, DESKTOP-3ADPNV0 resolves to a loopback address: 127.0.1.1; using 172.29.57.13 instead (on interface eth0)
22/08/19 00:23:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/19 00:23:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/19 00:23:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.formula.api import ols, glm

In [3]:
yellow_spark = spark.read.parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/yellow_credit')

In [4]:
yellow_spark.limit(10)

tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,fare_amount,tip_amount,total_amount,day_of_week,is_weekend,trip_length,Start(ET),Attendance,Win,margin_victory/loss
2018-12-25 00:39:10,2018-12-25 01:04:33,2.56,162,263,16.5,4.58,22.88,Tue,false,25.3833,12:00,19812,No,-14
2018-12-25 09:08:51,2018-12-25 09:13:27,0.9,163,162,5.0,1.45,7.25,Tue,false,4.6,12:00,19812,No,-14
2018-12-25 10:29:03,2018-12-25 10:31:40,0.63,100,48,4.0,0.96,5.76,Tue,false,2.6167,12:00,19812,No,-14
2018-12-25 10:58:19,2018-12-25 11:14:08,8.08,161,13,23.5,0.0,24.3,Tue,false,15.8167,12:00,19812,No,-14
2018-12-25 15:52:23,2018-12-25 16:07:03,2.13,162,142,11.5,1.0,13.3,Tue,false,14.6667,12:00,19812,No,-14
2018-12-25 00:31:15,2018-12-25 00:42:04,2.88,230,236,11.0,2.36,14.16,Tue,false,10.8167,12:00,19812,No,-14
2018-12-25 00:51:53,2018-12-25 00:58:03,0.98,161,229,6.0,1.36,8.16,Tue,false,6.1667,12:00,19812,No,-14
2018-12-25 03:53:56,2018-12-25 04:06:21,1.64,246,107,9.5,0.0,10.3,Tue,false,12.4167,12:00,19812,No,-14
2018-12-25 17:03:49,2018-12-25 17:07:48,0.83,237,141,5.0,2.0,7.8,Tue,false,3.9833,12:00,19812,No,-14
2018-12-25 17:38:54,2018-12-25 17:42:43,1.1,237,236,5.5,1.58,7.88,Tue,false,3.8167,12:00,19812,No,-14


In [5]:
yellow_weekday = yellow_spark.filter( F.col("is_weekend") == True )
yellow_weekend = yellow_spark.filter( F.col("is_weekend") == False)

yellow_win = yellow_spark.filter( F.col("win") == "Yes" )
yellow_lose = yellow_spark.filter( F.col("win") == "No" )



In [6]:
yellow_win.count()

1338679

In [7]:
yellow_lose.count()

5123011

In [20]:
from pyspark.sql.functions import *
aggregate_month = (yellow_spark
             .groupBy(month("tpep_pickup_datetime").alias("month"))
             .agg(round(mean("tip_amount"),4).alias("mean_tip"),
                 round(mean("trip_distance"),4).alias("mean_distance"),
                 round(mean("fare_amount"),4).alias("mean_fare"),
                 round(mean("total_amount"),4).alias("mean_total"),
                 round(mean("trip_length"),4).alias("mean_length"),
                 round(mean("margin_victory/loss"),4).alias("mean_margin"),
                 round(mean("Attendance"),4).alias("mean_attendance"))
              .orderBy("month")
)
aggregate_day.show(24)

+---+--------+-------------+---------+----------+-----------+-----------+---------------+
|day|mean_tip|mean_distance|mean_fare|mean_total|mean_length|mean_margin|mean_attendance|
+---+--------+-------------+---------+----------+-----------+-----------+---------------+
|  1|  2.1808|       1.9505|  10.0336|   14.1025|    11.9441|    -2.1304|     19013.4593|
|  3|  2.2224|        2.065|   9.9969|   14.4964|    11.3412|    -7.2285|     18305.3494|
|  5|  2.2998|       1.9739|  10.1839|   14.9446|    12.2229|    -7.0531|     18823.8292|
|  7|  2.3962|       2.1722|   9.9516|   15.8067|    10.8308|        3.0|        19812.0|
|  8|  2.1107|       1.9784|  10.5353|   13.6814|    12.9662|       -8.0|        18662.0|
|  9|  2.3008|        2.038|  10.0794|   15.1052|    11.7183|     -8.143|     19118.8947|
| 10|  2.5281|       2.0038|  10.2646|   16.4624|    12.2903|      -26.0|        19812.0|
| 11|  2.0775|       1.9957|  10.1002|   13.3512|    11.8496|   -19.9415|        19812.0|
| 13|  2.2

In [34]:
# average tip amount per hour in a weekday
(yellow_weekday
 .groupBy(hour("Pickup_hourly").alias("hour"))
 .agg(mean("tip_amount").alias("mean_tip"))
 .show()
)

+----+------------------+
|hour|          mean_tip|
+----+------------------+
|  12|2.1626819055201585|
|  22| 2.446235691321408|
|   1|2.3254755988302573|
|  13| 2.176139068634324|
|   6| 2.139035560671228|
|  16|2.2606291177283406|
|   3| 2.274831516671488|
|  20|2.3287920516273064|
|   5| 2.312550354345394|
|  19| 2.264033660544166|
|  15| 2.224087137265075|
|  17| 2.281596101551816|
|   9| 2.006334670956713|
|   4|2.3428525984354143|
|   8| 2.012010099542251|
|  23|2.4537930505652645|
|   7|2.0283621560006453|
|  10| 2.062244875098028|
|  21|2.3846844674469443|
|  11|2.1112948192322296|
+----+------------------+
only showing top 20 rows



In [35]:
# average tip amount per hour in a weekend

(yellow_weekend
 .groupBy(hour("Pickup_hourly").alias("hour"))
 .agg(mean("tip_amount").alias("mean_tip"))
 .show()
)


+----+------------------+
|hour|          mean_tip|
+----+------------------+
|  12|2.2458609367016087|
|  22| 2.465508696308898|
|   1|2.3667451764857774|
|  13|  2.23097526874565|
|   6|1.8982083076804819|
|  16| 2.278479554981979|
|   3| 2.370431875677628|
|  20| 2.342019114867856|
|   5| 2.078286411128841|
|  19|2.3225182269978872|
|  15|2.1823695085714725|
|  17| 2.313739749237615|
|   9| 2.204921297798007|
|   4|2.4315203257371736|
|   8| 2.126849155621941|
|  23|2.4685516834425147|
|   7| 2.008542537977298|
|  10|2.2460774457931643|
|  21| 2.421188850021614|
|  11| 2.278307719669036|
+----+------------------+
only showing top 20 rows

